In [12]:
class SHELF :
    def __init__(self) :
        self.products = []
    
    def push(self, product) :
        self.products.append(product)
        
    def pop(self, product) :
        # 꺼낼 물건이 선반에 없는 경우는 현재 고려하지 않음
        for i in range(len(self.products)) :
            if self.products[i].code == product.code :
                del self.products[i]
                return
    # 선반에 물건이 몇개 들어있는 알려준다.
    def stuffed(self) :
        return len(self.products)
    
    #데이터 보여주기용 함수
    def show(self) :
        if len(self.products) == 0 :
            print("EMPTY")
        for product in self.products :
            print(product)
        
        
    
class PRODUCT :
    #  현재 단계에서 잘모르는 데이터에 대해서는 defalut 값으로 처리함
    def __init__(self, code, name, ABC = 1, x = 0, y = 0, z = 0, g = 0) :
        self.code = code
        self.name = name
        self.ABC = ABC
        self.x = x
        self.y = y
        self.z = z
        self.g = g
    def __repr__(self) :
        return f"code = {self.code}, name = {self.name}"
        

class MAP :
    def __init__(self,R=2,C=5,K=987654321) :
        if R < 1 or C < 1: 
            raise ValueError("공장 크기가 적절하지 않습니다.")
        R *= 10
        R += 10
        C *= 8
        self.R = R
        self.C = C
        self.K = K
        self.idx = 1
        self.map = [[0]*C for _ in range(R)]
        self.products_in_shelves = {}
        
        # 워크스테이션 설정
        for c in range(3,C,8) :
            self.map[3][c] = -1
            self.map[R-4][c] = -1
            
        # 선반 설정
        break_point = False
        for c in range(1,self.C-2,4) :
            if break_point : break
            for r in range(7,self.R-10,10) :
                if break_point : break
                for dc in range(2) :
                    if break_point : break
                    for dr in range(6) :
                        cr = r + dr
                        cc = c + dc 
                        self.map[cr][cc] = self.idx
                        self.idx += 1
                        if self.idx > K :
                            break_point = True
                            break
        
        # 선반 배열 생성
        self.shelves = [SHELF() for _ in range(self.idx)]

        
    # 물건을 적재할 인덱스가 주어지지 않은 경우 비어있는 선반의 인덱스를 찾는다.
    def find_free_shelf(self) :
        for i in range(1,self.K+1) :
            if self.shelves[i].stuffed() < 60 :
                return i
            
    # 해당 인덱스를 가진 선반에 물건을 넣는다.
    def push(self, product, idx = 0) :
        if idx == 0 :
            Full = True
            for i in range(1,self.idx) :
                if self.shelves[i].stuffed() < 60 :
                    idx = i
                    Full = False
                    break
            if Full :
                print("모든 선반에 물건이 차있기 때문에 적재할 수 없습니다.")
                return
#         if 
#         self.shelves[idx].push(product)
    
    # 해당 인덱스를 가진 선반에서 물건을 뺀다.
    def pop(self, product, idx) :
        self.shelves[idx].pop(product)
             
    # 해당 인덱스를 가진 선반이 어떤 물건을 가지고 있는지 보여준다.
    def show_nth_shelf(self, idx) :
        print(f"{idx} 번째 선반에 들어있는 상품 : ")
        self.shelves[idx].show()

    
    def __repr__(self):
        ret = ''
        ret += f"R : {self.R} C : {self.C}\n"
        for r in range(self.R) :
            for c in range(self.C) :
                ret += str(self.map[r][c]) + " "*(len(str(self.idx)) - len(str(self.map[r][c])) + 1)
            ret += '\n'
        return ret
    
    # 선반과 워크스테이션만을 보여주는 직관적인 뷰
    def show(self) :
        print(f"R : {self.R} C : {self.C}\n")
        for r in range(self.R) :
            for c in range(self.C) :
                if self.map[r][c] == -1 :
                    print('W', end = ' ')
                elif self.map[r][c] > 0 :
                    print("@", end = ' ')
                else : print('-', end = ' ')
            print("\n")

In [13]:
M = MAP()
# 상품 적재 - 인풋 : 적재할 상품, 적재할 선반의 인덱스
M.push(PRODUCT("001", "치약"))
# 적재된 상품 확인 - 인풋 : 확인할 선반의 인덱스  ~ 데이터 확인용 함수
M.show_nth_shelf(1)

# 상품 추출 - 인풋 : 추출할 상품, 해당 상품이 있는 인덱스
M.pop(PRODUCT("001", "치약"), 1)
# 추출되었는 확인
M.show_nth_shelf(1)

1 번째 선반에 들어있는 상품 : 
code = 001, name = 치약
1 번째 선반에 들어있는 상품 : 
EMPTY


In [3]:
# 사용 예시
# test 수정

# 맵 생성 - 인풋 : (공장의 가로, 공장의 세로, 선반의 개수), + 타입 (필요할 경우)
M = MAP(7,10,20)
# 선반 모양 확인
print("선반 모양 : \n", M, sep = '')

# 상품 적재 - 인풋 : 적재할 상품, 적재할 선반의 인덱스
M.push(PRODUCT("001", "치약"), 1)
# 적재된 상품 확인 - 인풋 : 확인할 선반의 인덱스  ~ 데이터 확인용 함수
M.show_nth_shelf(1)

# 상품 추출 - 인풋 : 추출할 상품, 해당 상품이 있는 인덱스
M.pop(PRODUCT("001", "치약"), 1)
# 추출되었는 확인
M.show_nth_shelf(1)

선반 모양 : 
R : 80 C : 80
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  
0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  0  0  0  -1 0  0  0  0  
0  0  0  0  0

In [115]:
import random
random.choices(['N','Y'], weights=[2,98])

['Y']